<a href="https://colab.research.google.com/github/deltorobarba/chemistry/blob/main/fermionic_hamiltonian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fermionic Hamiltonian with Second Quantization**

> Second quantization refers to a framework primarily used for describing quantum systems in terms of **creation and annihilation operators**, especially in **wavefunction-based methods** like **Coupled Cluster (CC)**, **Configuration Interaction (CI)**, and **Møller-Plesset Perturbation Theory (MP2)**.

For the first quantization the Extracting Kohn-Sham Hamiltonian was computed.  The molecular Hamiltonian in a form suitable for quantum computing, e.g. second quantization is computed now.

We translate the molecular Hamiltonian into a form suitable for quantum computing (e.g., second quantization), you typically use a quantum chemistry package such as OpenFermion, which can help with the second quantization process.


Instead of energy functionals like B3LYP, methods like **MP2**, **CCSD (Coupled Cluster with Single and Double excitations)**, and **CI** rely on correlation functions or approximations to describe electron-electron interactions, but they don't rely on the exchange-correlation functionals from DFT. Functionals and second quantization are related in some contexts:

1. **Correlation Functionals in Post-Hartree-Fock Methods:**
   - In second quantization methods, **correlation energy** is described using a wavefunction-based approach rather than an energy functional like in DFT. For example, MP2 uses perturbation theory on top of the Hartree-Fock reference, and CCSD expands the wavefunction with coupled electron pairs.
   
2. **Coupled-Cluster DFT (CC-DFT):**
   - There are hybrid approaches, such as **CC-DFT**, where DFT-derived exchange-correlation energies are used alongside the wavefunction expansions of coupled-cluster methods. In these cases, DFT functionals (such as PBE, B3LYP) may be used to approximate parts of the electron correlation, although it's still not common to think of them as fully second-quantized approaches.

- In strict second-quantization approaches (e.g., **MP2**, **CC**, **CI**), functionals like those used in DFT are not typically used.
- Wavefunction-based methods are more about solving many-body quantum equations rather than using approximate energy functionals.
- Hybrid methods like **CC-DFT** represent an exception where functionals from DFT might influence parts of a second-quantization calculation.

In [ ]:
!pip install openfermion pyscf openfermionpyscf -q

In [ ]:
from pyscf import gto, scf, ao2mo
from openfermion import MolecularData, transforms
from openfermion.transforms import jordan_wigner
from openfermionpyscf import generate_molecular_hamiltonian
from openfermion.transforms import bravyi_kitaev

# Define the molecule
geometry = [
    ('O', (0.0000000, 0.0000000, 0.0000000)),
    ('H', (0.7586022, 0.0000000, 0.5042847)),
    ('H', (-0.7586022, 0.0000000, 0.5042847))
]
basis = 'cc-pvdz' # Smaller basis set (e.g., sto-3g instead of cc-pvdz) to reduce the number of orbitals involved.
charge = 0
multiplicity = 1

# Create a MolecularData object
molecule = MolecularData(geometry, basis, multiplicity, charge)

# Perform a Hartree-Fock calculation
mol = gto.M(
    atom=geometry,
    basis=basis,
    charge=charge,
    spin=multiplicity - 1
)

mf = scf.RHF(mol)
mf.kernel()

# Number of electrons and orbitals for the molecule
num_electrons = mol.nelectron
num_orbitals = mf.mo_coeff.shape[1]

# Manually extract integrals from PySCF
h1 = mf.get_hcore()  # One-electron integrals
eri = ao2mo.kernel(mol, mf.mo_coeff)  # Two-electron integrals
nuclear_repulsion = mol.energy_nuc()

# Use OpenFermion to generate the molecular Hamiltonian
molecular_hamiltonian = generate_molecular_hamiltonian(
    geometry=geometry,
    basis=basis,
    multiplicity=multiplicity,
    charge=charge,
    n_active_electrons=num_electrons,
    n_active_orbitals=num_orbitals
)

# Get the Hamiltonian in second-quantized form
fermion_hamiltonian = transforms.get_fermion_operator(molecular_hamiltonian)

# Convert the fermionic Hamiltonian to a qubit Hamiltonian using Jordan-Wigner transformation
qubit_hamiltonian = jordan_wigner(fermion_hamiltonian)
# qubit_hamiltonian = bravyi_kitaev(fermion_hamiltonian) # To speed up: Convert to qubit Hamiltonian using Bravyi-Kitaev transform

# Print the qubit Hamiltonian
print("Qubit Hamiltonian (Jordan-Wigner):\n", qubit_hamiltonian)

Streaming output truncated to the last 5000 lines.
(0.005675777528722197+0j) [Y26 X27 X34 Y35] +
(-0.0018167352815085828+0j) [Y26 X27 X34 Z35 Z36 Z37 Z38 Z39 Z40 Z41 Z42 Y43] +
(0.0008038738215991707+0j) [Y26 X27 X34 Z35 Z36 Z37 Z38 Z39 Z40 Z41 Z42 Z43 Z44 Y45] +
(0.0018167352815085828+0j) [Y26 X27 X35 Z36 Z37 Z38 Z39 Z40 Z41 Y42] +
(-0.0008038738215991707+0j) [Y26 X27 X35 Z36 Z37 Z38 Z39 Z40 Z41 Z42 Z43 Y44] +
(0.005288920978990235+0j) [Y26 X27 X36 Y37] +
(0.00044659976100352365+0j) [Y26 X27 X36 Z37 Z38 Z39 Z40 Z41 Z42 Z43 Z44 Z45 Z46 Y47] +
(-0.00044659976100352354+0j) [Y26 X27 X37 Z38 Z39 Z40 Z41 Z42 Z43 Z44 Z45 Y46] +
(0.0017967822570604877+0j) [Y26 X27 X38 Y39] +
(0.0032709477088930143+0j) [Y26 X27 X40 Y41] +
(0.009746300326900758+0j) [Y26 X27 X42 Y43] +
(-0.003834756732026591+0j) [Y26 X27 X42 Z43 Z44 Y45] +
(0.003834756732026591+0j) [Y26 X27 X43 Y44] +
(0.00555246099010602+0j) [Y26 X27 X44 Y45] +
(0.002458937387950069+0j) [Y26 X27 X46 Y47] +
(-0.004557340943066556+0j) [Y26 Y27 X2

**This can be computationally intensive because:**
* Number of Orbitals and Electrons: If your molecule has many orbitals (e.g., a large basis set) or electrons, the Hamiltonian will involve many terms. The Jordan-Wigner transformation processes each of these terms and converts them to qubit operators, which can take time.
* Two-Electron Integrals: These integrals contribute many terms to the Hamiltonian, and the mapping from fermions to qubits adds further complexity.

**To speed this up**:
* Try using a **smaller basis set** (e.g., sto-3g instead of cc-pvdz) to reduce the number of orbitals involved.
* **Active Space Approximation**: You can reduce the number of orbitals by focusing on an "active space" that includes only the most chemically relevant orbitals. For example, in OpenFermion-PySCF, you can define an active space of electrons and orbitals to restrict the number of orbitals used in the calculation.
* Use the **Bravyi-Kitaev transformation**: The Bravyi-Kitaev transformation may sometimes be faster and more efficient than the Jordan-Wigner transformation for certain systems, as it has better scaling for specific problem sizes.

**To speed up: Active Space Approximation and Smaller Basis Set**
* n_active_electrons = 4: Specifies the number of active electrons in the calculation. You can adjust this value based on your molecule's complexity and the level of approximation you're willing to make.
* n_active_orbitals = 6: Specifies the number of active orbitals, reducing the size of the problem.

In [ ]:
from pyscf import gto, scf, ao2mo
from openfermion import MolecularData, transforms
from openfermion.transforms import jordan_wigner
from openfermionpyscf import generate_molecular_hamiltonian

# Define the molecule
geometry = [
    ('O', (0.0000000, 0.0000000, 0.0000000)),
    ('H', (0.7586022, 0.0000000, 0.5042847)),
    ('H', (-0.7586022, 0.0000000, 0.5042847))
]
basis = 'sto-3g'  # Smaller basis set
charge = 0
multiplicity = 1

# Perform a Hartree-Fock calculation
mol = gto.M(
    atom=geometry,
    basis=basis,
    charge=charge,
    spin=multiplicity - 1
)

mf = scf.RHF(mol)
mf.kernel()

# Print the number of molecular orbitals and electrons
num_orbitals = mf.mo_coeff.shape[1]
num_electrons = mol.nelectron

print(f"Total number of orbitals: {num_orbitals}")
print(f"Total number of electrons: {num_electrons}")

# Set the active space (ensure these values are within the total available)
n_active_electrons = min(4, num_electrons)  # Adjust to be <= total number of electrons
n_active_orbitals = min(4, num_orbitals)    # Adjust to be <= total number of orbitals

# Use OpenFermion-PySCF to generate the molecular Hamiltonian with the active space
molecular_hamiltonian = generate_molecular_hamiltonian(
    geometry=geometry,
    basis=basis,
    multiplicity=multiplicity,
    charge=charge,
    n_active_electrons=n_active_electrons,
    n_active_orbitals=n_active_orbitals
)

# Get the Hamiltonian in second-quantized form
fermion_hamiltonian = transforms.get_fermion_operator(molecular_hamiltonian)

# Convert the fermionic Hamiltonian to a qubit Hamiltonian using Jordan-Wigner transformation
qubit_hamiltonian = jordan_wigner(fermion_hamiltonian)

# Print the qubit Hamiltonian
print("Qubit Hamiltonian (Jordan-Wigner):\n", qubit_hamiltonian)

converged SCF energy = -74.9458810293525
Total number of orbitals: 7
Total number of electrons: 10
Qubit Hamiltonian (Jordan-Wigner):
 (-72.99825826924977+0j) [] +
(-0.014191365186105021+0j) [X0 X1 Y2 Y3] +
(-0.013909319431865326+0j) [X0 X1 Y4 Y5] +
(-0.01615186881667349+0j) [X0 X1 Y6 Y7] +
(0.014191365186105021+0j) [X0 Y1 Y2 X3] +
(0.013909319431865326+0j) [X0 Y1 Y4 X5] +
(0.01615186881667349+0j) [X0 Y1 Y6 X7] +
(-0.001070287399482069+0j) [X0 Z1 X2 X3 Z4 X5] +
(-0.001070287399482069+0j) [X0 Z1 X2 Y3 Z4 Y5] +
(-0.034520910653578624+0j) [X0 Z1 Z2 Z3 X4] +
(-0.01054577598094817+0j) [X0 Z1 Z2 Z3 X4 Z5] +
(-0.023463011247503055+0j) [X0 Z1 Z2 Z3 X4 Z6] +
(-0.009569323556085469+0j) [X0 Z1 Z2 Z3 X4 Z7] +
(0.013893687691417586+0j) [X0 Z1 Z2 Z3 Z4 X5 Y6 Y7] +
(-0.013893687691417586+0j) [X0 Z1 Z2 Z3 Z4 Y5 Y6 X7] +
(-0.02506995711080215+0j) [X0 Z1 Z2 X4] +
(-0.026140244510284218+0j) [X0 Z1 Z3 X4] +
(-0.026888819761779426+0j) [X0 Z2 Z3 X4] +
(0.014191365186105021+0j) [Y0 X1 X2 Y3] +
(0.01390931943